# Setup

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

# 1.  inference

명령어

Head 3을 사용할 때 
* python detect.py --weights runs/train/yolov5s_new/weights/best.pt --source data/images 
* python detect.py --weights runs/train/yolov5s_new/weights/best.pt --source data/video

Head 4를 사용할 때 
* python detect.py --weights runs/train/yolov5s_new/weights/best.pt --source data/images
* python detect.py --weights runs/train/yolov5s_new/weights/best.pt --source data/video

## 1.1 Scene-specific method

File: /Detect_yolov5/utils/function.py
아래의 파일은 function.py의 line 12 ~ 52을 나타낸다.

크게 헤드를 4개(Nano, Small, Medium, Large)를 사용한 것과 헤드 3개(Small, Medium, Large)를 사용한 두가지 방법으로 나뉜다.

헤드 4개 및 3개를 사용하기 위해 설정해야하는 코드는 다음과 같다.

1. head_stages = [ ]
2. start, end, head = num, num, num

In [5]:



#head_stages = [4, 6, 10, 13, 14, 17, 20, 23]  # if you want head 3 
head_stages = [2, 4, 6, 10, 13, 14, 17, 18, 21, 24, 27, 30] # if you want head 4 

start, end, head = 22, 30, 1 #start of cut, end of cut, How many will you use to head

# 네트워크 차단
def convert(q, w, r):    
        # if head 3
        if q == 18 and w ==23 and r == 1:
            out = [17]
            print("only 17(80x80) head")
            return out, 0, 1
        elif q == 21 and w ==23 and r == 2:
            out = [17, 20]
            print("17(80x80) head and 20(40x40) head")
            return out, 0, 2
        elif q == 0 and w ==0 and r == 0:
            out = [17, 20, 23]
            print("default")
            return out, 0, 3
        
        # if head 4 
        elif q == 22 and w ==30 and r == 1:
            out = [21]
            print("21(160x160)")
            return out, 0, 1
        elif q == 25 and w ==30 and r == 2:
            out = [21, 24]
            print("21(160x160) and 24(80x80)")
            return out, 0, 2      
        elif q == 28 and w ==30 and r == 3:
            out = [21, 24, 27]
            print("21(160x160) and 24(80x80)head and 27(40x40)head")
            return out, 0, 3
        elif q == 0 and w ==0 and r == 1:
            out = [21, 24, 27, 30]
            print("default of head 4")
            return out, 0, 4
        
        else:
            [print("\n\n[Please check to command in function of convert (yolo.py line 42)]\n\n") for x in range(5)]  

### head 3의 사용 예시
- head 3을 사용할 경우
1.  ``` head_stages = [4, 6, 10, 13, 14, 17, 20, 23]  # if you want head 3 ``` 를 주석 해제하고 ```head_stages = [2, 4, 6, 10, 13, 14, 17, 18, 21, 24, 27, 30] # if you want head 4 ``` 를 주석처리
2.  ```start, end, head = 0, 0, 0 ``` 사용자가 환경에 적합한 Head를 선택하여 블록처리
- - ex) ```start, end, head = 0, 0, 0```     <-- 기본모델 헤드 3개 사용(Small, Medium, Large)
- - ex) ```start, end, head = 18, 23, 1```   <-- 헤드 1개 사용(Small)
- - ex) ```start, end, head = 21, 23, 2```   <-- 헤드 2개 사용(Small, Medium)

### head 4의 사용 예시
- head 4를 사용할 경우
1.  ``` head_stages = [4, 6, 10, 13, 14, 17, 20, 23]  # if you want head 3 ``` 를 주석 처리하고 ```head_stages = [2, 4, 6, 10, 13, 14, 17, 18, 21, 24, 27, 30] # if you want head 4 ``` 를 주석해제
2.  ```start, end, head = 0, 0, 1 ``` 사용자가 환경에 적합한 Head를 선택하여 블록처리
- - ex) ```start, end, head = 0, 0, 1``` <-- 헤드 4개 사용(Nane, Small, Medium, Large)
- - ex) ```start, end, head = 22, 30, 1``` <-- 헤드 1개 사용(Small)
- - ex) ```start, end, head = 25, 30, 2``` <-- 헤드 2개 사용(Small, Medium)
- - ex) ```start, end, head = 28, 30, 3``` <-- 헤드 3개 사용(Small, Medium, Large)

## 1.2 Knowledge distillation
상위모델의 결과값을 통해 하위모델을 훈련시키기 위한 방법으로 상위모델의 추론결과를 txt 파일로 저장하여 저장된 txt 파일을 통해 하위모델을 훈련시킨다.

File: /Detect_yolov5/extract.py
아래의 파일은 extract.py의 line 148 ~ 196을 나타낸다.

명령어 
* python extract.py --weights runs/train/yolov5s_new/weights/best.pt --source data/images --nosave

In [ ]:
#label 경로설정
            label_path = "C:/Users/dlwlsgh/Detect_yolov5/runs/ext/"

            if len(det):
                # 파일 이름 변환
                rename1 = p.name
                new_rename = rename1.replace('.png', '').replace('.jpg', '')
                # 파일 입력
                file = open(f"{label_path}{new_rename}.txt", "w+") 
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                   

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh  앵커박스 위치
                        #C_xywh = str(xywh)[1:-1] # 괄호만 없애기
                        C_xywh = ' '.join(map(str, xywh)) #list에서 , : 양쪽 괄호 없애기 
                        c = int(cls)  # integer class 클래스 종류 번호
                        #print(f"interger class 클래스 종류 번호: {c}")
                        
                        # 필요한 클래스만  추출
                        if 0 <= c <= 80: #'person_0', 'bicycle_1', 'car_2',motorcycle_3
                           file.write(f"{str(c)} {C_xywh} \n") # 텍스트 파일 입력
                        # if  c ==7 or 2 : 
                        #    file.write(f"{str(1)} {C_xywh}\n") 
                        # # elif c == 2 or c == 5 or c == 7 : #bus_5 #truck_7
                        #     file.write(f"{str(0)} {C_xywh}\n")
                        # elif c == 2:
                        #     file.write(f"{str(0)} {C_xywh}\n")

                        #print(f"{c} {C_xywh}\n") # 클래스 번호 및 좌표
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                        if save_crop:
                         save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)
                file.close()


사용하기 위한 방법은 다음과 같다.

1. txt 파일이 저장될 경로를 설정. 

2. 추출하고 싶은 클래스만 설정하여 사용한다.


### 1. txt 파일이 저장될 경로를 설정한다. 
-  첫번째로 선언된 label_path에 txt 파일을 저장할 폴더를 설정한다.
-  ```label_path = "path"  ```
-  - ex) ```label_path = "C:/Users/dlwlsgh/Detect_yolov5/runs/ext/"```

In [9]:
            #label 경로설정
            label_path = "C:/Users/dlwlsgh/Detect_yolov5/runs/ext/"

### 2. 추출하고 싶은 클래스만 설정하여 사용한다.
-  기본적으로 YOLOv5의 사전 훈련된 모델은 80개의 클래스를 가지고 있다. 하지만 본연구에서는 보행자, 차량만 사용하면 되므로 다른 클래스들은 무시하고 추출(보행자, 차량)하여 오탐지에 대한 부정확성을 줄인다.

- 파이썬 사전 훈련된 클래스 종류 <br><br>
- - ```[0, 'person', 1, 'bicycle', 2, 'car', 3, 'motorcycle', 4, 'airplane', 5, 'bus', 6, 'train', 7, 'truck', 8, 'boat', 9, 'traffic light', 10, 'fire hydrant', 11, 'stop sign', 12, 'parking meter', 13, 'bench', 14, 'bird', 15, 'cat', 16, 'dog', 17, 'horse', 18, 'sheep', 19, 'cow', 20, 'elephant', 21, 'bear', 22, 'zebra', 23, 'giraffe', 24, 'backpack', 25, 'umbrella', 26, 'handbag', 27, 'tie', 28, 'suitcase', 29, 'frisbee', 30, 'skis', 31, 'snowboard', 32, 'sports ball', 33, 'kite', 34, 'baseball bat', 35, 'baseball glove', 36, 'skateboard', 37, 'surfboard', 38, 'tennis racket', 39, 'bottle', 40, 'wine glass', 41, 'cup', 42, 'fork', 43, 'knife', 44, 'spoon', 45, 'bowl', 46, 'banana', 47, 'apple', 48, 'sandwich', 49, 'orange', 50, 'broccoli', 51, 'carrot', 52, 'hot dog', 53, 'pizza', 54, 'donut', 55, 'cake', 56, 'chair', 57, 'couch', 58, 'potted plant', 59, 'bed', 60, 'dining table', 61, 'toilet', 62, 'tv', 63, 'laptop', 64, 'mouse', 65, 'remote', 66, 'keyboard', 67, 'cell phone', 68, 'microwave', 69, 'oven', 70, 'toaster', 71, 'sink', 72, 'refrigerator', 73, 'book', 74, 'clock', 75, 'vase', 76, 'scissors', 77, 'teddy bear', 78, 'hair drier', 79, 'toothbrush'] ``` <br><br><br>

- 아래의 코드는 사람은 클래스 0으로 그대로 출력하고, 자동차, 버스, 트럭은 클래스 1로 새로 오버랩 하여 사용하는 예제를 나타낸다. 


In [ ]:
                        # 필요한 클래스만  추출
                        if c == 0 : # 사람은 그대로 클래스 0 으로 출력
                           file.write(f"{str(c)} {C_xywh} \n") 
                        elif c == 2 or c == 5 or c == 7 : # 자동차와 트럭의 경우 클래스 1로 출력
                            file.write(f"{str(0)} {C_xywh}\n")

- 추출된 txt 파일을 

# 3. Train

## Head 4개 모델 사용

기존 yolov5에서 제공하는 yolov5s 모델이 아닌 헤드 및 스테이지를 추가한 head4 모델을 사용한다. 
수정된 부분은 각 단계별 stage 및 Nano 크기의 헤드가 추가 되었고,  

 anchor는 auto-anchor가 사용 되므로 anchor box의 사전 설정은 사용하지 않는다.

File: /Detect_yolov5/models/yolov5s.yaml 과 File: /Detect_yolov5/models/yolov5s_new.yaml
아래의 파일은 욜로에서 제공되는 기본 모델(head3)인 yolov5s.yaml 파일과 수정할 모델인(head4)yolov5s_new.yaml 파일을 나타낸다.

### head 3 모델(base)

In [ ]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from: 어느 계층에서 오는 모듈 입력인지, number: 모듈이 반복되는 횟수, module: 모듈 이름, args: 모듈 인수]
  # [from, number, module, args]
  # chnal kernal stride p
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]], 
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large) 

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5) # 24
  ]

### head4 모델

In [ ]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 3

# YOLOv5 v6.0 backbone
backbone:
  # [from: 어느 계층에서 오는 모듈 입력인지, number: Conv모듈의  Bottleneck이 반복되는 횟수, module: 모듈 이름, args: 모듈 인수]
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

  
head:
  [[-1, 1, Conv, [512, 1, 1]], # 10
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],  # 11
   [[-1, 6], 1, Concat, [1]],  # 12 cat backbone P4 
   [-1, 3, C3, [512, False]],  # 13 

   [-1, 1, Conv, [256, 1, 1]], # 14
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],  # 15
   [[-1, 4], 1, Concat, [1]],  # 16 cat backbone P3 
   [-1, 3, C3, [256, False]],  # 17 
   
   [-1, 1, Conv, [128, 1, 1]], # 18
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],  # 19
   [[-1, 2], 1, Concat, [1]],  # 20 cat backbone P2 
   [-1, 3, C3, [128, False]],  # 21 (P2/4-nano)

   [-1, 1, Conv, [128, 3, 2]],  # 22
   [[-1, 18], 1, Concat, [1]],   # 23 cat head P3 
   [-1, 3, C3, [256, False]],  # 24 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],  # 25
   [[-1, 14], 1, Concat, [1]],  # 26 cat head P4
   [-1, 3, C3, [512, False]],  # 27 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],  # 28
   [[-1, 10], 1, Concat, [1]],  # 29 cat head P5
   [-1, 3, C3, [1024, False]],  # 30 (P5/32-large) #False ==shortcut을 의미

   [[21, 24, 27, 30], 1, Detect, [nc, anchors]],  # Detect(P2, P3, P4, P5)
  ]


### 훈련 명령어 head 4

In [ ]:
# commend
!python train.py --img 640 --batch 16 --epochs 100 --data aict.yaml --weights yolov5s.pt --cfg yolov5s_new.yaml

### Tensorboard로 모델 성능 확인

In [ ]:
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train